## Grepping functions from the vulnerability context of the file.

# Merging output generated by FlawFinder and CppCheck

In [23]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import lizard
import subprocess as sub
from pylibsrcml import srcml
import os
import xml.etree.ElementTree as et 
import sys
import csv
# from lxml import etree
import pandas as pd
import subprocess as sub
import sys
from io import BytesIO, StringIO
import numpy as np
import itertools
import importlib


# add parent dir to access modules
sys.path.append('../')

import src.tools as src
importlib.reload(src)

# from src.tools import apply_flawfinder, apply_cppcheck, apply_rats
# from src.scanner import  merge_flawfinder_cppcheck, project_flaws

# # CppCheck scaning a dir
chk_dir = '../data/projects/contiki-2.4/'

df_ff = src.apply_flawfinder(chk_dir)
# df_cc = apply_cppcheck(chk_dir)
# df_rat = apply_rats(chk_dir)
# df_flaw = merge_flawfinder_cppcheck(df_ff, df_cc, df_rat)
df_ff

flawfinder --csv --inputs ../data/projects/contiki-2.4/


,File,Line,Column,DefaultLevel,Level,Category,Name,Warning,Suggestion,Note,CWEs,Context,Fingerprint,ToolVersion,RuleId,HelpUri
0,Error: encoding error in ../data/projects/cont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,'utf-8' codec can't decode byte 0xf6 in positi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Python3 requires input character data to be pe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,it also requires perfectly correct system enco...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Unfortunately,your data and/or system settings are not.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Here are some options:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1. Run: PYTHONUTF8=0 python3 flawfinder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,if your system and and data are all properl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,a non-UTF-8 encoding.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2. Run: PYTHONUTF8=0 LC_ALL=C.ISO-2022 python3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


  Preparing metadata (setup.py) ... done
  Created wheel for importlib: filename=importlib-1.0.4-py3-none-any.whl size=5852 sha256=34fa1ea5f90de3f1f345505925bd8f1a54e79a12aa3710683972a9f27f3bbabc
  Stored in directory: /Users/guru/Library/Caches/pip/wheels/bb/1e/4d/b9e2665ab36264acf947eab5715b2e1ab4fd1649adcb96785d
Successfully built importlib
